# 道路

In [3]:
import pandas as pd

In [4]:
data = pd.DataFrame(data=[
    [1, 2000, 4000],
    [2, 3000, 5000],
    [3, 1500, 3000],
    [4, 2200, 4300],
    [5,  500, 1000],
    [6, 1500, 1500],
    [7, 2500, 2500],
    [8,  100,  300],
    [9,  600, 1000],
    [10,1000, 2000],
    ],
    columns=['プロジェクト', '支出', '現在価値']).set_index('プロジェクト')
data

,支出,現在価値
プロジェクト,,
1,2000,4000
2,3000,5000
3,1500,3000
4,2200,4300
5,500,1000
6,1500,1500
7,2500,2500
8,100,300
9,600,1000


## 定式化
$i$番目のプロジェクトの採択の有無をバイナリ変数$x_i$（採択：1 不採択：0）、現在価値を$b_i$、支出を$c_i$とする。<br>
最適化の定式化は、
\begin{align}
\max_{\bf x}&\sum_{i=1}^mb_ix_i\\
s.t. &\sum_{i=1}^{10}c_ix_i\leq 5000\\
&\sum_{i=1}^4x_i\leq 1\\
&\sum_{i=5}^7x_i\leq 1\\
&\sum_{i=8}^{10}x_i\leq 1
\end{align}
さらに追加の制約は
\begin{align}
x_2=x_6\\
x_4=x_7
\end{align}
となる。

In [32]:
from pulp import *
m = LpProblem(sense=LpMaximize) # 最大化問題として定式化
x = [LpVariable('x{0}'.format(i+1), cat=LpBinary) for i in range(len(data))] # 0-1の変数xの設定
m += (data['現在価値']*x).sum() # 目的関数
m += (data['支出']*x).sum() <= 5000 # 制約条件
m += sum(x[0:4]) <= 1 # 制約条件
m += sum(x[4:7]) <= 1 # 制約条件
m += sum(x[7:11]) <= 1 # 制約条件
m += x[1] == x[5]
m += x[3] == x[6]

In [33]:
m.solve() # ソルバーの実行

1

In [34]:
ans = [bool(value(x[i])) for i in range(len(data))]
project = data[ans]
project.loc['合計'] = project.sum(axis=0)
project

,支出,現在価値
プロジェクト,,
4,2200,4300
7,2500,2500
8,100,300
合計,4800,7100


よって新しい追加条件の元ではプロジェクト1, 2, 3 が採択される。